In [4]:
import pandas as pd
from classes.clustering import investigate_messages_OpenStack
from datetime import datetime
import numpy as np
import re
import torch

In [2]:
import create_dataloaders

True

In [9]:
df = pd.read_csv('../data/OpenStack_data_original.csv', names=['real_index','line_id','timestamp','hour','pid','level','service','Content','test_id','parent_service','round_1','round_2','api_round_1','api_round_2','assertions_round_1','assertions_round_2','time_hour','clusters','round','EventId','EventTemplate','ParameterList','anom_label','encoded_labels'
],)
# df2 = pd.read_pickle('../data/OpenStack_data_original.csv_structured.pickle')
# df_templates = pd.read_csv('../data/OpenStack_data_original.csv_templates.csv')

In [35]:
log_format = '<real_index>,<line_id>,<timestamp>,<hour>,<pid>,<level>,<service>,<Content>,<test_id>,<parent_service>,<round_1>,<round_2>,<api_round_1>,<api_round_2>,<assertions_round_1>,<assertions_round_2>,<time_hour>,<clusters>,<round>,<EventId>,<EventTemplate>,<ParameterList>,<anom_label>,<encoded_labels>'

In [15]:
df = pd.read_csv('../data/clusters/300s_clusters_30_.csv')

In [16]:
df

,service,Content,level,service.1,round_1,round_2,api_round_1,api_round_2,assertions_round_1,assertions_round_2,...,test_id,parent_service,roun,lenf,pred_1,pred_2,pred_3,pred_4,pred_5,pred_kmeans
0,[],[],[],[],[],[],[],[],[],[],...,1,cinder,1,0,0,0,0,0,0,0
1,['[sample_workload.sh]:' '[sample_workload.sh]...,['Workload started!'\n 'Create image tempest-c...,['INFO' 'INFO' 'INFO' 'INFO' 'INFO' 'INFO' 'IN...,['[sample_workload.sh]:' '[sample_workload.sh]...,['FAILURE' 'FAILURE' 'FAILURE' 'FAILURE' 'FAIL...,['NO_FAILURE' 'NO_FAILURE' 'NO_FAILURE' 'NO_FA...,['not_applicable' 'not_applicable' 'not_applic...,['not_applicable' 'not_applicable' 'not_applic...,['FAILURE_VOLUME_ATTACHED' 'FAILURE_VOLUME_ATT...,['NO_FAILURE' 'NO_FAILURE' 'NO_FAILURE' 'NO_FA...,...,1,foreground,1,43,1,1,1,1,1,1
2,['eventlet.wsgi.server' 'eventlet.wsgi.server'],"['""""""POST v2 images HTTP 1.1"""" 201 867 0.3097...",['INFO' 'INFO'],['eventlet.wsgi.server' 'eventlet.wsgi.server'],['FAILURE' 'FAILURE'],['NO_FAILURE' 'NO_FAILURE'],['not_applicable' 'not_applicable'],['not_applicable' 'not_applicable'],['FAILURE_VOLUME_ATTACHED' 'FAILURE_VOLUME_ATT...,['NO_FAILURE' 'NO_FAILURE'],...,1,glance,1,2,4,4,4,4,4,4
3,['keystone.common.wsgi' 'keystone.common.wsgi'...,['GET 10.0.20.24:35357 ' 'GET 10.0.20.24:353...,['INFO' 'INFO' 'INFO' 'INFO' 'INFO' 'INFO' 'IN...,['keystone.common.wsgi' 'keystone.common.wsgi'...,['FAILURE' 'FAILURE' 'FAILURE' 'FAILURE' 'FAIL...,['NO_FAILURE' 'NO_FAILURE' 'NO_FAILURE' 'NO_FA...,['not_applicable' 'not_applicable' 'not_applic...,['not_applicable' 'not_applicable' 'not_applic...,['FAILURE_VOLUME_ATTACHED' 'FAILURE_VOLUME_ATT...,['NO_FAILURE' 'NO_FAILURE' 'NO_FAILURE' 'NO_FA...,...,1,keystone,1,65,17,17,17,17,17,17
4,['neutron.wsgi' 'neutron.wsgi' 'neutron.wsgi' ...,"['""10.0.20.24 """"GET v2.0 security-groups HTTP...",['INFO' 'INFO' 'INFO' 'INFO' 'INFO' 'INFO' 'IN...,['neutron.wsgi' 'neutron.wsgi' 'neutron.wsgi' ...,['FAILURE' 'FAILURE' 'FAILURE' 'FAILURE' 'FAIL...,['NO_FAILURE' 'NO_FAILURE' 'NO_FAILURE' 'NO_FA...,['not_applicable' 'not_applicable' 'not_applic...,['not_applicable' 'not_applicable' 'not_applic...,['FAILURE_VOLUME_ATTACHED' 'FAILURE_VOLUME_ATT...,['NO_FAILURE' 'NO_FAILURE' 'NO_FAILURE' 'NO_FA...,...,1,neutron,1,14,3,3,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1689,['nova.api.openstack.wsgi' 'nova.osapi_compute...,"[""HTTP exception thrown: Impossibile trovare l...",['INFO' 'INFO' 'INFO' 'WARNING' 'INFO' 'INFO' ...,['nova.api.openstack.wsgi' 'nova.osapi_compute...,['NO_FAILURE' 'NO_FAILURE' 'NO_FAILURE' 'NO_FA...,['NO_FAILURE' 'NO_FAILURE' 'NO_FAILURE' 'NO_FA...,['not_applicable' 'not_applicable' 'not_applic...,['not_applicable' 'not_applicable' 'not_applic...,['NO_FAILURE' 'NO_FAILURE' 'NO_FAILURE' 'NO_FA...,['NO_FAILURE' 'NO_FAILURE' 'NO_FAILURE' 'NO_FA...,...,434,nova,1,41,25,25,25,25,25,25
1690,['neutron.agent.securitygroups_rpc' 'neutron.w...,"[' Remove device filter for '\n '""10.0.20.24 ""...",['INFO' 'INFO' 'INFO' 'INFO' 'INFO' 'INFO' 'IN...,['neutron.agent.securitygroups_rpc' 'neutron.w...,['NO_FAILURE' 'NO_FAILURE' 'NO_FAILURE' 'NO_FA...,['NO_FAILURE' 'NO_FAILURE' 'NO_FAILURE' 'NO_FA...,['not_applicable' 'not_applicable' 'not_applic...,['not_applicable' 'not_applicable' 'not_applic...,['NO_FAILURE' 'NO_FAILURE' 'NO_FAILURE' 'NO_FA...,['NO_FAILURE' 'NO_FAILURE' 'NO_FAILURE' 'NO_FA...,...,434,neutron,1,22,3,3,3,3,3,3
1691,['nova.api.openstack.placement.requestlog'\n '...,"['""10.0.20.24 """"GET placement allocations 0ff...",['INFO' 'INFO' 'INFO' 'INFO' 'INFO' 'INFO' 'IN...,['nova.api.openstack.placement.requestlog'\n '...,['NO_FAILURE' 'NO_FAILURE' 'NO_FAILURE' 'NO_FA...,['NO_FAILURE' 'NO_FAILURE' 'NO_FAILURE' 'NO_FA...,['not_applicable' 'not_applicable' 'not_applic...,['not_applicable' 'not_applicable' 'not_applic...,['NO_FAILURE' 'NO_FAILURE' 'NO_FAILURE' 'NO_FA...,['NO_FAILURE' 'NO_FAILURE' 'NO_FAILURE' 'NO_FA...,...,434,nova,2,64,3,3,3,3,3,3
1692,['neutron.agent.

In [18]:
def string_to_list(x):
        """Converts a string like '[1 2 3 5]' to a np.array

        """
        filter_spaces = re.sub(r' +', ',', x)
        filter_newlines = filter_spaces.replace('\n', '')
        if filter_newlines == '[]':
            return np.array([]).astype('object')
        output_list = []
        for el in filter_newlines[1:-1].split(','):
            if el != '':
                output_list.append(el)
        return np.array(output_list).astype('object')

In [19]:
df.round_1.apply(string_to_list)

0                                                      []
1       ['FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', '...
2                                  ['FAILURE', 'FAILURE']
3       ['FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', '...
4       ['FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', '...
                              ...                        
1689    ['NO_FAILURE', 'NO_FAILURE', 'NO_FAILURE', 'NO...
1690    ['NO_FAILURE', 'NO_FAILURE', 'NO_FAILURE', 'NO...
1691    ['NO_FAILURE', 'NO_FAILURE', 'NO_FAILURE', 'NO...
1692    ['FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', '...
1693    ['FAILURE', 'FAILURE', 'FAILURE', 'FAILURE', '...
Name: round_1, Length: 1694, dtype: object